In [1]:
import os
import pandas as pd
from src import download
from src.core import Proposicao

In [ ]:
projs = pd.read_excel(r"./projetos_selecionados.xlsx")

for idx in projs.index:
    prop = Proposicao(
        sigla_tipo=projs.loc[idx, "tipo"],
        numero=projs.loc[idx, "código"],
        ano=projs.loc[idx, "ano "],
    )

    resp = download.get_projects_by_name([prop])

    arvore = download.get_arvore_apensados(resp)

    autores = download.get_autores(arvore.nodes)

    props_detalhes = download.get_proposicoes_detalhes(arvore.nodes)

    votacoes = download.get_votacoes(arvore.nodes)

    votos = download.get_votos(votacoes)

    nome_tema = projs.loc[idx, "tema "]

    path = f"./data/{nome_tema}"
    try:
        os.mkdir(path)
    except FileExistsError:
        pass

    pd.DataFrame(props_detalhes).to_csv(path + "/projetos.csv")
    pd.DataFrame(autores).to_csv(path + "/autores.csv")
    pd.DataFrame(votacoes).to_csv(path + "/votacoes.csv")
    pd.DataFrame(votos).to_csv(path + "/votos.csv")
